In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
import pandas as pd
from PIL import Image
from sklearn.metrics import f1_score
import numpy as np
from timm import create_model

# Define the dataset class
class AUDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transform
        self.data['fname'] = self.data['fname'].astype(str)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.data.iloc[idx]['fname'])
        image = Image.open(img_name).convert("RGB")
        labels = torch.tensor(self.data.iloc[idx, 3:].values.astype('float32'))
        if self.transform:
            image = self.transform(image)
        return image, labels

# Define the model with Dropout
class AUModel(nn.Module):
    def __init__(self, num_aus):
        super(AUModel, self).__init__()
        self.base_model = create_model('deit_small_patch16_224', pretrained=True)
        self.base_model.head = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(self.base_model.head.in_features, num_aus)
        )

    def forward(self, x):
        return self.base_model(x)

# Training function
def train_model(model, train_loader, val_loader, test_loader, optimizer, criterion, num_epochs=6):
    best_val_f1 = 0.0
    best_model_state = None

    for epoch in range(num_epochs):
        model.train()
        train_preds, train_labels = [], []
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_preds.extend(torch.sigmoid(outputs).cpu().detach().numpy())
            train_labels.extend(labels.cpu().numpy())

        val_preds, val_labels = [], []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                val_preds.extend(torch.sigmoid(outputs).cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        val_f1 = f1_score(np.array(val_labels) > 0.5, np.array(val_preds) > 0.5, average='macro')
        print(f"Epoch {epoch+1}/{num_epochs}, Val F1: {val_f1:.4f}")

        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            best_model_state = model.state_dict()

    model.load_state_dict(best_model_state)
    return model

# File paths
train_csv_files = [
    'drive/My Drive/TIF/train_new/TIF_AUG_AU_SI_ver1_fold_0.csv',
    'drive/My Drive/TIF/train_new/TIF_AUG_AU_SI_ver1_fold_1.csv',
    'drive/My Drive/TIF/train_new/TIF_AUG_AU_SI_ver1_fold_2.csv',
]
test_csv_files = [
    'drive/My Drive/TIF/val_new/TIF_AUG_AU_SI_ver1_fold_0.csv',
    'drive/My Drive/TIF/val_new/TIF_AUG_AU_SI_ver1_fold_1.csv',
    'drive/My Drive/TIF/val_new/TIF_AUG_AU_SI_ver1_fold_2.csv',
]
image_dir = 'drive/My Drive/TIF/CroppedFromPhotos/TIF_DB_Augmented'

# Define Action Unit columns
au_columns = ['AU1', 'AU2', 'AU3', 'AU4', 'AU6', 'AU9', 'AU12', 'AU20']
num_aus = len(au_columns)

# Data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

criterion = nn.BCEWithLogitsLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

all_predictions = []

for train_csv, test_csv in zip(train_csv_files, test_csv_files):
    full_train_dataset = AUDataset(train_csv, image_dir, transform=transform)
    test_dataset = AUDataset(test_csv, image_dir, transform=transform)
    train_size = int(0.8 * len(full_train_dataset))
    val_size = len(full_train_dataset) - train_size
    train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    model = AUModel(num_aus).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    model = train_model(model, train_loader, val_loader, test_loader, optimizer, criterion, num_epochs=6)

    test_preds = []
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            outputs = torch.sigmoid(model(inputs))
            test_preds.extend(outputs.cpu().numpy() > 0.5)

    fold_predictions = pd.DataFrame(test_preds, columns=au_columns)
    fold_predictions['fname'] = test_dataset.data['fname'].iloc[:len(test_preds)].values
    all_predictions.append(fold_predictions)

final_predictions = pd.concat(all_predictions, ignore_index=True)
final_predictions.to_csv('all_test_predictions.csv', index=False)

print("Training complete. Predictions saved.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/88.2M [00:00<?, ?B/s]

Epoch 1/6, Val F1: 0.6113


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import torch
import numpy as np

# Confusion matrix display function for subplot
def plot_confusion_matrix_subplot(true_labels, predictions, classes, ax):
    cm = confusion_matrix(true_labels, predictions)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    disp.plot(cmap=plt.cm.Blues, ax=ax)
    ax.set_title(f'Confusion Matrix')

# Test phase to collect predictions and true labels
test_preds, test_labels = [], []
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = torch.sigmoid(model(inputs))  # Sigmoid output for multi-label classification
        preds = (outputs > 0.5).cpu().numpy()  # Apply threshold of 0.5 for binary classification

        # Collect predictions and true labels
        test_preds.extend(preds)
        test_labels.extend(labels.cpu().numpy())

# Convert to numpy arrays for sklearn compatibility
test_preds = np.array(test_preds)
test_labels = np.array(test_labels)

# Set up subplot grid (4 columns)
num_classes = test_preds.shape[1]
ncols = 4
nrows = (num_classes + ncols - 1) // ncols  # Calculate number of rows needed

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16, nrows * 4))
axes = axes.flatten()  # Flatten the axes array for easier indexing

# Plot confusion matrices in subplots
for i in range(num_classes):
    ax = axes[i]
    plot_confusion_matrix_subplot(test_labels[:, i], test_preds[:, i], classes=[0, 1], ax=ax)

    # Adjust titles for each subplot
    ax.set_title(f'Class: {au_columns[i]}')

# Remove any unused subplots
for j in range(num_classes, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np

# Compute metrics for each Action Unit (AU)
results = []  # To store metrics for each AU

for i in range(test_preds.shape[1]):  # Loop through each AU
    true_positive = np.sum((test_labels[:, i] == 1) & (test_preds[:, i] == 1))  # True positives
    false_positive = np.sum((test_labels[:, i] == 0) & (test_preds[:, i] == 1))  # False positives
    false_negative = np.sum((test_labels[:, i] == 1) & (test_preds[:, i] == 0))  # False negatives
    true_negative = np.sum((test_labels[:, i] == 0) & (test_preds[:, i] == 0))  # True negatives

    total_samples = true_positive + false_positive + false_negative + true_negative  # Total samples for the AU

    # Calculate Recall (True Positive Rate)
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0

    results.append({
        "AU": au_columns[i],
        "Total Samples": total_samples,
        "Correctly Predicted": true_positive + true_negative,
        "Wrong Prediction": false_positive + false_negative,
        "Recall (%)": recall * 100  # Recall as a percentage
    })

# Convert results to a DataFrame for better visualization
results_df = pd.DataFrame(results)

# Print the results as a table
print(results_df)

# Optionally, save the results to a CSV file
results_df.to_csv("AU_metrics_total.csv", index=False)

In [ ]:
from sklearn.metrics import f1_score

# Flatten arrays for global metrics
flat_true_labels = test_labels.ravel()
flat_pred_labels = test_preds.ravel()

# Calculate overall F1 score (macro-average across all AUs)
overall_f1 = f1_score(flat_true_labels, flat_pred_labels, average='macro')

# Display the overall F1 score
print(f"Overall F1 Score (Macro-Average): {overall_f1:.4f}")


In [ ]:
import pandas as pd

# Train CSV dosyalarının yollarını listeleyin
train_csv_files = [
    'drive/My Drive/TIF/train_new/TIF_AUG_AU_SI_ver1_fold_0.csv',
    'drive/My Drive/TIF/train_new/TIF_AUG_AU_SI_ver1_fold_1.csv',
    'drive/My Drive/TIF/train_new/TIF_AUG_AU_SI_ver1_fold_2.csv',
]

# Tüm CSV'leri birleştirin

train_data = pd.concat([pd.read_csv(csv_file) for csv_file in train_csv_files], ignore_index=True)

# Duplike satırları kaldırın
train_data_cleaned = train_data.drop_duplicates()

# Yeni temizlenmiş veriyi kaydedin
train_data_cleaned.to_csv('combined_train_data_tif.csv', index=False)

print(f"Orijinal veri sayısı: {len(train_data)}, Temizlenmiş veri sayısı: {len(train_data_cleaned)}")


In [ ]:
pred = pd.read_csv("all_test_predictions_with_val.csv")
actual = pd.read_csv("combined_train_data_tif.csv")
pred = pred.replace({True: 1, False: 0})
columns = ['fname'] + [col for col in pred.columns if col != 'fname']
pred = pred[columns]
pred.to_csv('pred.csv', index=False)

In [ ]:
predictions = pd.read_csv("pred.csv")
common_rows = pd.merge(predictions, actual)
print(f"Common rows: {len(common_rows)}")

In [ ]:
final_predictions.to_csv('/content/drive/My Drive/TIF/all_test_predictions_with_val.csv', index=False)
